# Analysis of the results

In [1]:
import os

In [2]:
def construct_path_from_parent(current_path, subdir_list):
    # Get the parent directory
    parent_dir = os.path.dirname(current_path)
    
    # Construct the path using os.path.join
    return os.path.join(parent_dir, *subdir_list)

In [3]:
current_dir = os.getcwd()
dataset_name = "kvasir"
model_name = "UNet_vanilla"
fold_nb = "fold_0"
subdirs = ["training", "outputs", dataset_name, model_name, fold_nb, "summary.json"]

summary_loc = construct_path_from_parent(current_dir, subdirs)
if os.path.exists(summary_loc):
    print(f'summary.json loc: {summary_loc}')
else:
    print(f'No file at the path you provided.')

summary.json loc: /home/rob/Documents/Github/seg-equi-architectures/src/Benchmarks/training/outputs/kvasir/UNet_vanilla/fold_0/summary.json
